<h1><center>Big Data Algorithms Techniques & Platforms</center></h1>
<h2>
<hr style=" border:none; height:3px;">
<center>Assignment 4 - MapReduce and Spark</center>
<hr style=" border:none; height:3px;">
</h2>

# Introduction


<p align="justify">
<font size="3">
In this exercise you is asked to use Spark for implementing an algorithm that applies computations on documents and dataframes.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Execute the following cell in order to initialize Spark**</font>
<hr style=" border:none; height:2px;">
</p>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar zxvf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"


import findspark
findspark.init()

#import of the SparkSession
from pyspark.sql import SparkSession

#inizialization of the Spark Session
spark = SparkSession \
    .builder \
    .appName("Assignment4") \
    .getOrCreate()

import pandas as pa

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

# Analysing documents


<p align="justify">
<font size="3">
We have already seen that MapReduce procedures are good in analyzing text-files.
    
The provided data comes from a scraping operation on the website https://www.vagalume.com.br/ and is available on kaggle:
    
https://www.kaggle.com/neisse
    

    
The assignment is divided in 2 parts:
    
* Part 1 is focused on MapReduce 
    
* Part 2  is focuses on dataframes
    </font>
    </p>
    
<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>Notice that  dataset is noisy and shows all the typical issues related with data coming from this procedure (duplicated entries, etc).</font>
<hr style=" border:none; height:2px;">
</p>

# Part 1 -  MapReduce
<p align="justify">
<font size="3">
In the provided folder you can find a set of documents/files containing  descriptions of songs (lyrics and additional informations). Specifically in each file:

- the first line is the idiom/language
- the second line is the relative url of the song of the original website
- the third line is the title of a song 
- from fourth line on the text you find the lyrics of the song.
    </font>
    </p>

## Exercise 1 - (2 points) - Song's lyrics 

<p align="justify">
<font size="3">
Provide a Spark MapReduce procedure that reads the documents and checks how many song's lyrics appear at least two times.

In the data-interpretation of this exercise you can consider that two files represent the same lyric if the url (3rd line of each file) is the same.

 </font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>Notice that  you can reuse any code that was made available for the previous labs/assignments or that you already developed in these contexts.</font>
<hr style=" border:none; height:2px;">
</p>

In [2]:
!unzip lyrics_files_idioms.zip

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  inflating: lyrics_files_idioms/51892.txt  
  inflating: lyrics_files_idioms/94270.txt  
  inflating: lyrics_files_idioms/144569.txt  
  inflating: lyrics_files_idioms/201933.txt  
  inflating: lyrics_files_idioms/3729.txt  
  inflating: lyrics_files_idioms/40747.txt  
  inflating: lyrics_files_idioms/135390.txt  
  inflating: lyrics_files_idioms/19632.txt  
  inflating: lyrics_files_idioms/143206.txt  
  inflating: lyrics_files_idioms/4046.txt  
  inflating: lyrics_files_idioms/47028.txt  
  inflating: lyrics_files_idioms/197731.txt  
  inflating: lyrics_files_idioms/27904.txt  
  inflating: lyrics_files_idioms/70684.txt  
  inflating: lyrics_files_idioms/105253.txt  
  inflating: lyrics_files_idioms/61951.txt  
  inflating: lyrics_files_idioms/50554.txt  
  inflating: lyrics_files_idioms/125183.txt  
  inflating: lyrics_files_idioms/84063.txt  
  inflating: lyrics_files_idioms/16501.txt  
  inflating: lyrics_files_id

In [2]:
### Write here your code

dir = './lyrics_files_idioms'

data = []
languages = []
urls = []
titles = []
lyrics = []

for filename in os.listdir(dir):
  route_file = dir + '/' + filename
  with open(route_file) as f:
    text = f.read()
    lines = text.splitlines()
    data.append(lines)
    languages.append(lines[0])
    urls.append(lines[1])
    titles.append(lines[2])
    lyrics.append(lines[3]) 


In [3]:
rdd_urls = spark.sparkContext.parallelize(urls)
rdd_urls = rdd_urls.map(lambda x: (x, 1)) \
             .reduceByKey(lambda x, y: x + y) \
             .filter(lambda x: x[1] >= 2) \
             .map(lambda x: ('duplicated', 1)) \
             .reduceByKey(lambda x, y: x + y)

rdd_urls.collect()

[('duplicated', 38096)]

## Exercise 2

### 2.1 - (1 point) - Distinct songs
Provide a Spark MapReduce procedure that provides how many distinct song's lyrics are present.

Also in this case consider the uri as key: two files represent the same lyric if the url is equal.

### 2.2 - (1 point) - Chaining
According to your implementation of Exercise 1, can you chain MapReduce additional MapReduce steps for solving Exercise 2.2? 

Provide the code for 2.1 and anwer for 2.2

In [4]:
### Write here your code followed by the answer to question 2.2

#  2.1 question - As stated by Francesca's e-mail:
#  "exercise 2,2 must count how many DISTINCT songs are present, 
#  the 2.1 must count how many songs appear twice."
rdd_urls_2 = spark.sparkContext.parallelize(urls)
rdd_urls_2 = rdd_urls_2.map(lambda x: (x, 1))\
              .reduceByKey(lambda x, y: x + y)

rdd_urls_21 = rdd_urls_2.filter(lambda x: x[1] == 2) \
              .map(lambda x: ('twice', 1))\
              .reduceByKey(lambda x, y: x + y)

print(rdd_urls_21.collect())

# 2.2 question
# Using the same base rdd of 2.1
rdd_urls_22 = rdd_urls_2.map(lambda x: ('distinct', 1))\
              .reduceByKey(lambda x, y: x + y)

print(rdd_urls_22.collect())


[('twice', 36125)]
[('distinct', 167499)]


# Exercise 3

### 3.1 - (3 points) - Most common word for language

Now that you discovered the duplicated documents consider just one occurence of each song's lyric and define a MapReduce procedure that finds the most common word for each language (of course you must remove stop words).




In [5]:
# Auxiliary Functions
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

regex = re.compile('[^a-zA-Z ]')

def remove_non_letters(word):
  return regex.sub('', word)

def preprocess(text, stopwords) :
  st = []
  for word in stopwords:
    st_i = remove_non_letters(word)
    st.append(st_i)
  words = text.map(lambda line: line.replace("'"," "))\
          .flatMap(lambda line: line.split(" "))\
          .map(lambda word: remove_non_letters(word))\
          .filter(lambda word: len(word) > 2).map(lambda word: word.lower())\
          .filter(lambda word: word not in st)
  return words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Extract the languages from the lyrics
rdd_languages = spark.sparkContext.parallelize(languages)
languages = rdd_languages.map(lambda x: (x, 1))\
            .reduceByKey(lambda x, y: x + y)\
            .sortBy(lambda x: x[1], ascending=False)
languages = languages.keys().collect()

# Divide the data in chunks
chunks = [data[x:x+50000] for x in range(0, len(data), 50000)]

# The lists with the relevant data from the map-reduce process.
empty_stopwords = []
common_words = []

# Iterate over the data
for chunk in chunks:
  data_d = chunk
  # Remove duplicate lyrics
  # Key-Values ((language,url), lyric)
  rdd_data = spark.sparkContext.parallelize(data_d)
  
  rdd_data = rdd_data.map(lambda x: ((x[0],x[1]), x[3]))\
           .reduceByKey(lambda x, y: x )
  
  # Iterate over the languages and process the data
  for lang in languages:
    # Filter the lyrics from the language
    rdd_lang = rdd_data.filter(lambda x: x[0][0] == lang)\
               .map(lambda x: x[1])
    lang_stopwords = []
    try:
      # Download language stopwords
      lang_stopwords = stopwords.words(lang.lower())
    except:
      empty_stopwords.append(lang.lower())
    finally:
      # Preprocessing steps: - Split lyric to words,
      #                      - Remove non-letters
      #                      - Transform to words with more than 2 chars
      #                      - Remove stopwords (if available)
      rdd_lang_p = preprocess(rdd_lang, lang_stopwords)
      stopwords_en = stopwords.words('english')
      rdd_lang_p = rdd_lang_p.filter(lambda word: word not in stopwords_en)
      
      # Map the preprocessed words, reduce them and get the most frequent
      rdd_lang_p = rdd_lang_p.map(lambda x: (x,1))\
                   .reduceByKey(lambda x, y: x + y)\
                   .sortBy(lambda x: x[1], ascending=False)
      
      # Add to the common_words list, the data from the chunks
      if rdd_lang_p.isEmpty() == False:
        list_lang_p = rdd_lang_p.take(20)
        for item in list_lang_p:
          common_words.append([lang, item[0], item[1]])

# Final list that will be returned
most_common_words = []

# Map-reduce to the top 20 words of each language-chunk
rdd_top = spark.sparkContext.parallelize(common_words)\
                .map(lambda x: ((x[0],x[1]),x[2]))\
                .reduceByKey(lambda x, y: x + y)

# Map-reduce and sorting to get the most used word
for lang in languages:
  top_word = rdd_top.filter(lambda x: x[0][0] == lang)\
              .map(lambda x: (x[0][1], x[1]))\
              .sortBy(lambda x: x[1], ascending=False).take(1)
  
  most_common_words.append([lang,top_word[0][0]])

# Final response
print('-'*25)
print('Most common word by language: ')
for i in most_common_words:
  print(i[0], ': \t', i[1])


-------------------------
Most common word by language: 
ENGLISH : 	 like
PORTUGUESE : 	 pra
SPANISH : 	 amor
NA : 	 nae
ITALIAN : 	 mai
FRENCH : 	 plus
GERMAN : 	 mehr
KINYARWANDA : 	 kimi
ICELANDIC : 	 sem
SWEDISH : 	 honey
FINNISH : 	 jotain
INDONESIAN : 	 get
ESTONIAN : 	 kuldseks
GALICIAN : 	 shake
DANISH : 	 forever
IRISH : 	 let
HAITIAN_CREOLE : 	 nou
BASQUE : 	 eta
CROATIAN : 	 koji
TAGALOG : 	 bebot
NORWEGIAN : 	 vestido
CATALAN : 	 bendita
SUNDANESE : 	 nanananananananananananana
DUTCH : 	 nimma
SWAHILI : 	 pouss
RUSSIAN : 	 mamaamastar
MALAY : 	 sing
SERBIAN : 	 nego
CEBUANO : 	 tibum
JAPANESE : 	 huh
ARABIC : 	 lejo
SESOTHO : 	 hae
NYANJA : 	 machika
KURDISH : 	 kevin
MALAGASY : 	 que
TURKISH : 	 loco
AFRIKAANS : 	 ons
BOSNIAN : 	 kad
ROMANIAN : 	 tenmmoce
SLOVAK : 	 uncoolohol
SLOVENIAN : 	 nekej
HUNGARIAN : 	 virgom
HMONG : 	 sento
KOREAN : 	 without
WELSH : 	 mae
GANDA : 	 ddara
POLISH : 	 czy
CZECH : 	 senta


### 3.2 - (3 points) - Most common end/start words

Finally discover, for each language, the most common ending and starting word (of course, also in this case) you must remove stop words).

In [7]:
def rev_sentence(sentence): 
  # first split the string into words
  words = sentence.split(' ') 
  reverse_sentence = ' '.join(reversed(words))
  return reverse_sentence 

In [8]:
# Extract the languages from the lyrics
rdd_languages = spark.sparkContext.parallelize(languages)
languages = rdd_languages.map(lambda x: (x, 1))\
            .reduceByKey(lambda x, y: x + y)\
            .sortBy(lambda x: x[1], ascending=False)
languages = languages.keys().collect()

data2 = []

for elem in data:
  lang = elem[0]
  url = elem[1]
  tit = elem[2]
  lyric = elem[3].split(" ")[0]
  rev_lyric = rev_sentence(elem[3]).split(" ")[0]
  data2.append([lang,url,tit,lyric,rev_lyric])

# Divide the data in chunks
chunks = [data2[x:x+50000] for x in range(0, len(data2), 50000)]

# The lists with the relevant data from the map-reduce process.
empty_stopwords = []
common_first = []
common_last = []

# Iterate over the data
for chunk in chunks:
  data_d = chunk
  # Remove duplicate lyrics
  # Key-Values ((language,url), lyric)
  rdd_data = spark.sparkContext.parallelize(data_d)
  
  rdd_data_first = rdd_data.map(lambda x: ((x[0],x[1]), x[3]))\
                  .reduceByKey(lambda x, y: x )
  
  rdd_data_last = rdd_data.map(lambda x: ((x[0],x[1]), x[4]))\
                  .reduceByKey(lambda x, y: x )

  # Iterate over the languages and process the data
  for lang in languages:
    # Filter the lyrics from the language
    rdd_first = rdd_data_first.filter(lambda x: x[0][0] == lang)\
                .map(lambda x: x[1])
    
    rdd_last = rdd_data_last.filter(lambda x: x[0][0] == lang)\
               .map(lambda x: x[1])

    lang_stopwords = []
    try:
      # Download language stopwords
      lang_stopwords = stopwords.words(lang.lower())
    except:
      empty_stopwords.append(lang.lower())
    finally:
      # Preprocessing steps: - Split lyric to words,
      #                      - Remove non-letters
      #                      - Transform to words with more than 2 chars
      #                      - Remove stopwords (if available)
      rdd_first = preprocess(rdd_first, lang_stopwords)
      rdd_last = preprocess(rdd_last, lang_stopwords)
      
      rdd_first = rdd_first.filter(lambda x: x not in ['intro','chorus','instrumental'])
      rdd_last = rdd_last.filter(lambda x: x not in ['intro','chorus','instrumental'])


      # Map the preprocessed words, reduce them and get the most frequent
      rdd_first = rdd_first.map(lambda x: (x,1))\
                  .reduceByKey(lambda x, y: x + y)\
                  .sortBy(lambda x: x[1], ascending=False)

      rdd_last = rdd_last.map(lambda x: (x,1))\
                  .reduceByKey(lambda x, y: x + y)\
                  .sortBy(lambda x: x[1], ascending=False)
      
      # Add to the common_first and common_last list, the data from the chunks
      if rdd_first.isEmpty() == False:
        list_first = rdd_first.take(20)
        for item in list_first:
          common_first.append([lang, item[0], item[1]])
      
      if rdd_last.isEmpty() == False:
        list_last = rdd_last.take(20)
        for item in list_last:
          common_last.append([lang, item[0], item[1]])

# Final list that will be returned
most_common_first_last = []

# Map-reduce to the top 20 words of each language-chunk (beginning and end)
rdd_first = spark.sparkContext.parallelize(common_first)\
            .map(lambda x: ((x[0],x[1]),x[2]))\
            .reduceByKey(lambda x, y: x + y)

rdd_last = spark.sparkContext.parallelize(common_last)\
           .map(lambda x: ((x[0],x[1]),x[2]))\
           .reduceByKey(lambda x, y: x + y)

# Map-reduce and sorting to get the most used word at the beginning and end
for lang in languages:
  top_first = rdd_first.filter(lambda x: x[0][0] == lang)\
              .map(lambda x: (x[0][1], x[1]))\
              .sortBy(lambda x: x[1], ascending=False).take(1)
  
  top_last  = rdd_last.filter(lambda x: x[0][0] == lang)\
              .map(lambda x: (x[0][1], x[1]))\
              .sortBy(lambda x: x[1], ascending=False).take(1)

  if lang == 'ARABIC' or lang == 'SESOTHO' or lang == 'RUSSIAN' or lang == 'NYANJA' or lang == 'MALAGASY' or lang == 'HMONG' or lang == 'SWAHILI' or lang == 'KOREAN' or lang == 'GANDA':
    most_common_first_last.append([lang, "NA", top_last[0][0]])
  elif lang == 'POLISH':
    most_common_first_last.append([lang,top_first[0][0], "NA"])
  else:
    most_common_first_last.append([lang,top_first[0][0], top_last[0][0]])

# Final response
print('-'*25)
print('Most common words (beginning and end) by language: ')
for i in most_common_first_last:
  print(i[0], ': \t \t (first): ', i[1], '\t (last): ', i[2])


-------------------------
Most common words (beginning and end) by language: 
ENGLISH : 	 	 (first):  well 	 (last):  yeah
SPANISH : 	 	 (first):  hoy 	 (last):  amor
NA : 	 	 (first):  hey 	 (last):  you
FRENCH : 	 	 (first):  quand 	 (last):  edamancio
GERMAN : 	 	 (first):  komm 	 (last):  rammstein
KINYARWANDA : 	 	 (first):  sora 	 (last):  oborenaiyo
ICELANDIC : 	 	 (first):  mir 	 (last):  hopelandic
SWEDISH : 	 	 (first):  regntunga 	 (last):  karusell
FINNISH : 	 	 (first):  taistella 	 (last):  vei
INDONESIAN : 	 	 (first):  dejar 	 (last):  berdua
ESTONIAN : 	 	 (first):  elus 	 (last):  lahuta
GALICIAN : 	 	 (first):  everybody 	 (last):  relato
DANISH : 	 	 (first):  forever 	 (last):  forever
IRISH : 	 	 (first):  tumbalacatumba 	 (last):  now
BASQUE : 	 	 (first):  eguzkiak 	 (last):  hil
CROATIAN : 	 	 (first):  krv 	 (last):  ref
CATALAN : 	 	 (first):  bendito 	 (last):  amor
SUNDANESE : 	 	 (first):  uyeonhi 	 (last):  geoya
RUSSIAN : 	 	 (first):  NA 	 (last):  amas


<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**DataFrames**</font>
<hr style=" border:none; height:2px;">
</p>

# Part 2 - Dataframes

In this part you can use Pandas Dataframes or  Spark Dataframes.  I suggest to use a Spark Dataframe
end exploit the Pandas functionalities as we have seen in the 2nd assignment. Download the two available datasets at the link:

https://www.kaggle.com/neisse/scrapped-lyrics-from-6-genres

You can find two .cvs files: 

* artists-data.csv

* lyrics-data.csv


# Import artist data.
<p align="justify">
<font size="3">
The artist data in the .csv file can be stored in a dataframe. 
    
Each row of the .csv file describes an artist and the columns represent the following data:
    
* Artist - The artist's name
* Popularity - Popularity score at the date of scrapping
* ALink - The link to the artist's page
* AGenre - Primary musical genre of the artist
* AGenres - A list (pay attention to the format) of genres the artist fits in
    
</font>
</p>


# Import song's lyrics data.


<p align="justify">
<font size="3">
    
Each row of the .csv file describes a lyric and the columns represent the following data:
    
* ALink - The link to the webpage of the artist
* SLink - The link to the webpage of the song
* Idiom - The idiom of the lyric
* Lyric - The lyrics
* SName - The name of the song

    

</font>
</p>


#  Exercise 4 - (3 points) - Artist's genre

<p align="justify">
<font size="3">
Provide a program that finds the artists for which the genre is not specified.

</font>
</p>






In [13]:
### Write here your code

from pyspark.sql.types import IntegerType, DoubleType, StringType 
from pyspark.sql.functions import col

# Loads the artist data from the csv file as a spark dataframe
dfs = spark.read.format("csv") \
      .option("header", True) \
      .load("artists-data.csv") \
      .withColumn("Artist",col("Artist").cast(StringType())) \
      .withColumn("Songs",col("Songs").cast(IntegerType())) \
      .withColumn("Popularity",col("Popularity").cast(DoubleType())) \
      .withColumn("Link",col("Link").cast(StringType())) \
      .withColumn("Genre",col("Genre").cast(StringType())) \
      .withColumn("Genres",col("Genres").cast(StringType()))

# Filters the dataframe for the null genres. All the Artists have a genre.
dfs.filter(dfs.Genre.isNull()).show()

+------+-----+----------+----+-----+------+
|Artist|Songs|Popularity|Link|Genre|Genres|
+------+-----+----------+----+-----+------+
+------+-----+----------+----+-----+------+



#  Exercise 5 - (3 points) - Duplicates

<p align="justify">
<font size="3">
Provide a program that removes the duplicates in the artists (also in this case the URL is the key).

</font>
</p>




In [17]:
### Write here your code

# dfs2 contains the artists without duplicate values
dfs2 = dfs.dropDuplicates(['Link'])

# This is a comparison between the dataframe with and without duplicate artists:
print("Before removing duplicates: ", dfs.count())
print("After removing duplicates: ", dfs2.count())


Before removing duplicates:  3242
After removing duplicates:  2940


#  Exercise 6 - (4 points)

<p align="justify">
<font size="3">
Provide a program that using dataframe return the 100 most popular artists and the lyrics of their songs.
</font>
</p>

In [27]:
### Write here your code

# Loads the lytics data from the csv file as a spark dataframe
dfs_lyric = spark.read.format("csv") \
            .option("header", True) \
            .load("lyrics-data.csv") \
            .withColumn("ALink",col("ALink").cast(StringType())) \
            .withColumn("SName",col("SName").cast(StringType())) \
            .withColumn("SLink",col("SLink").cast(StringType())) \
            .withColumn("Lyric",col("Lyric").cast(StringType())) \
            .withColumn("Idiom",col("Idiom").cast(StringType())) \

# Filters the 100 more popular artist
dfs_artist = dfs2.sort(dfs2.Popularity.desc()).take(100)
dfs_artist = spark.createDataFrame(dfs_artist)

# Renaming the spark dataframes that will be joined
dfs_artists = dfs_artist.alias('dfs_artists')
dfs_lyrics = dfs_lyric.alias('dfs_lyrics')

# Joining both dataframes with an inner join
df_results = dfs_artists.join(dfs_lyrics, dfs_artists.Link == dfs_lyrics.ALink, "inner")

# Dropping not used columns and sorting the songs by genre
df_results = df_results.drop("Songs","Link","Genres","ALink","SLink","Idiom")
df_results = df_results.sort(df_results.Popularity.desc())

# Printing in console the first 5 results
df_results.show(5)


+-------------+----------+-----+--------------------+--------------------+
|       Artist|Popularity|Genre|               SName|               Lyric|
+-------------+----------+-----+--------------------+--------------------+
|Ariana Grande|     246.8|  Pop|7 Rings (Feat. 2 ...|"Yeah, breakfast ...|
|Ariana Grande|     246.8|  Pop|       Thank U, Next|"Thought I'd end ...|
|Ariana Grande|     246.8|  Pop|               Needy|If you take too l...|
|Ariana Grande|     246.8|  Pop|             7 Rings|"Yeah, breakfast ...|
|Ariana Grande|     246.8|  Pop|Break Up With You...|"You got me some ...|
+-------------+----------+-----+--------------------+--------------------+
only showing top 5 rows



# 2 - Bonus 


<p align="justify">
<font size="3">
Using the approach you prefer (just Dataframes, hybrid approach)  :
    
* the 10 most common words in the lyrics of each artist
* the 10 most common words for each genre. For this question we can use the primary genre of the artist.

</font>
</p>




In [ ]:
# Write here your code and the detailed description of the MapReduce algorithm.




















